**Author**: J W Debelius<br>
**Date**: 25 October 2015<br>
**Virtualenv**: power\_play

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import skbio

import absloute_power.distance as dist

I'm going to simulate distances between locations which should be significantly different when the two categories are compared, apply a PERMANOVA and ANOSIM (cites needed), and then determine how to better evaluate effect size on microbiome data. 

In [2]:
def simulate_poisson_distance(length, scale, param):
    return np.random.poisson(param, length) * scale

In [221]:
np.random.poisson(3, 5)

array([3, 4, 3, 2, 4])

In [16]:
locations = np.random.binomial(1, 0.5, (30))
locations.sort()

In [19]:
within0 = 0.15
within1 = 0.2
between = 0.5

In [20]:
locations

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

In [25]:
# num0 = np.sum(locations == 0)
# num1 = np.sum(locations == 1)

In [55]:
in0 = np.random.uniform(0, 1, np.square(num0 - 1)/2)*within0

In [142]:
num0 = 4

In [174]:
num0 = 2
vec = np.arange((np.square(num0)-1)/2) + 1
dm0 = np.zeros((num0, num0))
print 'vec', vec

pos_count = 0
dm0 = np.zeros((num0, num0))
for i in xrange(num0-1):
    pos1 = np.arange(i+1, num0)
    print 'pos1', pos1
    pos2 = np.arange(i, num0-1) + pos_count
    print 'pos2', pos2
    pos_count = pos_count + len(pos2) - 1
    dm0[i, pos1] = vec[pos2]
    dm0[pos1, i] = vec[pos2]
print dm0

vec [1]
pos1 [1]
pos2 [0]
[[ 0.  1.]
 [ 1.  0.]]


In [179]:
num0 = 3
vec = np.arange((np.square(num0)-1)/2) + 1
dm0 = np.zeros((num0, num0))
print 'vec', vec

pos_count = 0
dm0 = np.zeros((num0, num0))
for i in xrange(num0-1):
    pos1 = np.arange(i+1, num0)
    print 'pos1', pos1
    pos2 = np.arange(i, num0-1) + pos_count
    print 'pos2', pos2
    pos_count = pos_count + len(pos2) - 1
    dm0[i, pos1] = vec[pos2]
    dm0[pos1, i] = vec[pos2]
print dm0

vec [1 2 3 4]
pos1 [1 2]
pos2 [0 1]
pos1 [2]
pos2 [2]
[[ 0.  1.  2.]
 [ 1.  0.  3.]
 [ 2.  3.  0.]]


In [181]:
num0 = 4
vec = np.arange((np.square(num0)-1)/2) + 1
dm0 = np.zeros((num0, num0))
print 'vec', vec

pos_count = 0
dm0 = np.zeros((num0, num0))
for i in xrange(num0-1):
    pos1 = np.arange(i+1, num0)
    pos2 = np.arange(i, num0-1) + pos_count
    pos_count = pos_count + len(pos2) - 1
    dm0[i, pos1] = vec[pos2]
    dm0[pos1, i] = vec[pos2]
print dm0

vec [1 2 3 4 5 6 7]
[[ 0.  1.  2.  3.]
 [ 1.  0.  4.  5.]
 [ 2.  4.  0.  6.]
 [ 3.  5.  6.  0.]]


In [239]:
def convert_to_dm(length, vec):
    """Converts a condensed distance matrix to a table format"""
    # Stacks the data
    vec = np.hstack(vec)
    # Creates the output matrix
    dm = np.zeros((length, length))
    # Adds a counter to watch the positon
    pos_count = 0
    
    # Populates the distance matrix
    for idx in xrange(length-1):
        pos1 = np.arange(idx+1, length)
        pos2 = np.arange(idx, length-1) + pos_count
        pos_count = pos_count + len(pos2) - 1
        dm[idx, pos1] = vec[pos2]
        dm[pos1, idx] = vec[pos2]
    
    return dm

Now, I need to be able to simulate distance matrix vectors.

In [253]:
num0 = 4
num1 = 4
num_d = num0 * num1

In [259]:
vec0 = np.arange(0, (num0/2)*(num0-1)) + 11
vec1 = np.arange(0, (num1/2)*(num1-1)) + 21
veci = np.arange(0, num0 * num1) + 101
print vec0
print vec1
print veci

[11 12 13 14 15 16]
[21 22 23 24 25 26]
[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]


In [328]:
vec0 = np.array([1, 2, 3])
vec1 = np.array([10, 20, 30])

In [329]:
dm0 = convert_to_dm(3, vec0)
dm1 = convert_to_dm(3, vec1)

In [330]:
print dm0
print dm1

[[ 0.  1.  2.]
 [ 1.  0.  3.]
 [ 2.  3.  0.]]
[[  0.  10.  20.]
 [ 10.   0.  30.]
 [ 20.  30.   0.]]


In [331]:
dmi = np.array([[11, 21, 31],
                [12, 22, 32],
                [13, 23, 33],
                ])

dmi

array([[11, 21, 31],
       [12, 22, 32],
       [13, 23, 33]])

In [336]:
dmi[::-1]

array([[13, 23, 33],
       [12, 22, 32],
       [11, 21, 31]])

In [332]:
np.vstack([np.hstack([dm0, dmi]),
          [np.hstack([])]])

array([[  0.,   1.,   2.,  11.,  21.,  31.],
       [  1.,   0.,   3.,  12.,  22.,  32.],
       [  2.,   3.,   0.,  13.,  23.,  33.]])